In [ ]:
!pip install -q openai-whisper pydub ffmpeg-python gradio soundfile


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 10.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.9 MB/s eta 

In [ ]:
import whisper
from google.colab import files
import os

# Load the Whisper model
model = whisper.load_model("base")

# Upload the audio file
uploaded = files.upload()

# Process each uploaded file
for filename in uploaded.keys():
    print(f"Transcribing {filename}...")

    # Perform transcription
    result = model.transcribe(filename)

    # Get the transcribed text
    transcribed_text = result["text"]

    # Save the transcribed text to a .txt file
    output_filename = filename.split('.')[0] + "_transcription.txt"
    with open(output_filename, "w") as file:
        file.write(transcribed_text)

    print(f"Transcription saved to {output_filename}")

    # Provide the file for download
    files.download(output_filename)


Saving RNN_nlp.mp3 to RNN_nlp.mp3
Transcribing RNN_nlp.mp3...


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning:

FP16 is not supported on CPU; using FP32 instead



Transcription saved to RNN_nlp_transcription.txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install -U jiwer
!pip show jiwer
import jiwer
import difflib
import html
import io
from io import BytesIO
import base64
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import numpy as np
import seaborn as sns
import plotly.express as px
from plotly.io import to_html
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display
from sklearn.metrics import confusion_matrix
from jiwer import compute_measures, process_words
from jiwer import (
    Compose,
    ToLowerCase,
    ExpandCommonEnglishContractions,
    RemoveKaldiNonWords,
    RemoveWhiteSpace,
    RemoveMultipleSpaces,
    RemovePunctuation,
    Strip,
    ReduceToSingleSentence,
    ReduceToListOfListOfWords,
)

wer_standardize_contiguous = Compose([
    ToLowerCase(),
    ExpandCommonEnglishContractions(),
    RemoveKaldiNonWords(),
    RemoveWhiteSpace(replace_by_space=True),
    RemoveMultipleSpaces(),
    RemovePunctuation(),
    Strip(),
    ReduceToSingleSentence(),
    ReduceToListOfListOfWords(),
])


# === Read files ===
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

# Input text paths (replace these with actual paths as needed)
generated_text = read_file('/content/RNN_nlp_transcription.txt')
ground_truth = read_file('/content/RNN_nlp.txt')

ground_truth_transformed = wer_standardize_contiguous(ground_truth)[0]
generated_text_transformed = wer_standardize_contiguous(generated_text)[0]
print(ground_truth_transformed)
print(generated_text_transformed)
# === Compute WER and classification elements ===
measures = compute_measures(ground_truth, generated_text, truth_transform=wer_standardize_contiguous, hypothesis_transform=wer_standardize_contiguous)
wer_score = measures["wer"]
substitution_count = measures['substitutions']
deletion_count = measures['deletions']
insertion_count = measures['insertions']
n_ref_words = measures.get('ref_len', len(ground_truth.split()))

# Derived metrics
TP = n_ref_words - substitution_count - deletion_count
FP = insertion_count + substitution_count
FN = deletion_count + substitution_count
accuracy = TP / (TP + FP + FN) if (TP + FP + FN) else 0
precision = TP / (TP + FP) if (TP + FP) else 0
recall = TP / (TP + FN) if (TP + FN) else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0

# === Highlight errors using jiwer.process_words ===
ref_words = ground_truth.split()
hyp_words = generated_text.split()

alignment = process_words(" ".join(ground_truth_transformed), " ".join(generated_text_transformed))
print (alignment)
# Flatten alignment chunks
flat_chunks = []
for entry in alignment.alignments:
  if isinstance(entry, list):
      flat_chunks.extend(entry)
  else:
      flat_chunks.append(entry)
print(flat_chunks)
highlighted_generated = []
highlighted_ground = []

for chunk in flat_chunks:
    ref = ref_words[chunk.ref_start_idx : chunk.ref_end_idx]
    hyp = hyp_words[chunk.hyp_start_idx : chunk.hyp_end_idx]
    ref_text = " ".join(map(html.escape, ref))
    hyp_text = " ".join(map(html.escape, hyp))
    if chunk.type == "equal":
        highlighted_generated.append(f"<span>{hyp_text}</span>")
        highlighted_ground.append(f"<span>{ref_text}</span>")
    elif chunk.type == "substitute":
        highlighted_generated.append(f"<span class='substitution'>{hyp_text}</span>")
        highlighted_ground.append(f"<span class='substitution'>{ref_text}</span>")
    elif chunk.type == "insert":
        print(chunk)
        highlighted_generated.append(f"<span class='insertion'>{hyp_text}</span>")
        highlighted_ground.append(f"<span></span>")  # nothing in truth
    elif chunk.type == "delete":
        highlighted_generated.append(f"<span></span>")  # nothing in generated
        highlighted_ground.append(f"<span class='deletion'>{ref_text}</span>")

print(highlighted_generated)
print(highlighted_ground)

highlighted_generated_html = " ".join(highlighted_generated)
highlighted_ground_html = " ".join(highlighted_ground)

# WER components mapped to confusion matrix quadrants
TN = 0
hits = TP
substitutions = substitution_count
insertions = insertion_count
deletions = deletion_count

# Create confusion matrix layout
matrix = np.array([[TN, FP],
                   [FN, TP]])

fig = go.Figure(data=go.Heatmap(
    z=matrix,
    x=['Predicted Negative', 'Predicted Positive'],
    y=['Actual Negative', 'Actual Positive'],
    text=[[f'TN: 0', f'FP: {substitutions + insertions}'],
          [f'FN: {substitutions + deletions}', f'TP: {hits}']],
    texttemplate='%{text}',
    colorscale='Blues',
    showscale=True,
    hoverongaps=False))

fig.update_layout(
#    title='Confusion Matrix with TP, TN, FP, FN and WER Components',
    xaxis_title='Predicted',
    yaxis_title='Actual',
    yaxis_autorange='reversed'
)
conf_matrix_html = fig.to_html(full_html=False, include_plotlyjs='cdn')

def get_color_code(value):
    if value >= 0.8:
        return "green"
    elif value >= 0.5:
        return "orange"
    else:
        return "red"

def get_wer_color(value):
    if value <= 0.2:
        return "green"
    elif value <= 0.5:
        return "orange"
    else:
        return "red"

# === WER Metrics Section ===
#<!-- WER Metrics Section -->
confusionmatrix_metrics_html = f"""
<div class="wer-container">
  <h2 class="wer-title">WER Metrics Results</h2>
  <div class="wer-metrics">
    <div class="wer-card">
      <p class="wer-label">Substitutions</p>
      <p class="wer-value">{substitutions}</p>
    </div>
    <div class="wer-card">
      <p class="wer-label">Insertions</p>
      <p class="wer-value">{insertions}</p>
    </div>
    <div class="wer-card">
      <p class="wer-label">Deletions</p>
      <p class="wer-value">{deletions}</p>
    </div>
    <div class="wer-card">
      <p class="wer-label">Correct Matches</p>
      <p class="wer-value">{hits}</p>
    </div>
  </div>
</div>
"""
# === Updated Metric Explanation HTML ===
wer_metrics_html = f"""
<h3>📐 Metrics (with Results)</h3>
<table>
    <thead>
        <tr><th>Confusion Matrix Labels</th><th>Interpreted as</th><th>Value</th></tr>
    </thead>
    <tbody>
        <tr><td>False Positive (FP)</td><td>Substitutions + Insertions</td><td>{insertions+substitutions}</td></tr>
        <tr><td>False Negative (FN)</td><td>Substitutions + Deletions</td><td>{deletions+substitutions}</td></tr>
        <tr><td>True Positive (TP)</td><td>Hits</td><td>{TP}</td></tr>
        <tr><td>True Negative (TN)</td><td>TN are not present in WER Analysis</td><td>{TN}</td></tr>

    </tbody>
</table>
"""
# Call get_color_code for each metric before using in the template
accuracy_color = get_color_code(accuracy)
precision_color = get_color_code(precision)
recall_color = get_color_code(recall)
f1_color = get_color_code(f1_score)

html_template = f"""
<div class="metrics-section">
  <h2>📊 Evaluation Results</h2>
  <div class="card-container">
    <div class="metric-card">
      <h3>🎯 Accuracy</h3>
      <p>TP / (TP + FP + FN)</p>
      <div class="metric-value" style="color: {accuracy_color}">{accuracy:.2%}</div>
    </div>

    <div class="metric-card">
      <h3>⚖️ Precision</h3>
      <p>TP / (TP + FP)</p>
      <div class="metric-value" style="color: {precision_color}">{precision:.2%}</div>
    </div>

    <div class="metric-card">
      <h3>🔍 Recall</h3>
      <p>TP / (TP + FN)</p>
      <div class="metric-value" style="color: {recall_color}">{recall:.2%}</div>
    </div>

    <div class="metric-card">
      <h3>💡 F1 Score</h3>
      <p>2 × (Precision × Recall) / (Precision + Recall)</p>
      <div class="metric-value" style="color: {f1_color}">{f1_score:.2%}</div>
    </div>
  </div>
</div>
"""
html = html_template.format(
    accuracy=accuracy,
    precision=precision,
    recall=recall,
    f1_score=f1_score,
    accuracy_color=get_color_code(accuracy),
    precision_color=get_color_code(precision),
    recall_color=get_color_code(recall),
    f1_color=get_color_code(f1_score)
)

# WER without normalization

wer_raw = jiwer.wer(generated_text, ground_truth)

# === Simplified WER Comparison HTML ===
wer_formula_html = f"""
<div style="font-size: 18px; line-height: 1.8;">
  <h3>WER Comparison</h3>
  <p><strong>WER (Without Normalization):</strong>
     <span style="color:{get_wer_color(wer_raw)}">{wer_raw:.2%}</span></p>
  <p><strong>WER (With Normalization):</strong>
     <span style="color:{get_wer_color(wer_score)}">{wer_score:.2%}</span></p>
</div>
"""

# === Pie Chart ===
labels = ['Substitutions', 'Deletions', 'Insertions']
sizes = [substitution_count, deletion_count, insertion_count]
colors = ['#fff176', '#e57373', '#81c784']

fig, ax = plt.subplots()
ax.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
ax.axis('equal')

buf = io.BytesIO()
plt.savefig(buf, format='png')
buf.seek(0)
img_base64 = base64.b64encode(buf.read()).decode('utf-8')
buf.close()
plt.close()
img_html = f'<img src="data:image/png;base64,{img_base64}" alt="WER Pie Chart" style="max-width:100%;">'

# Derived values for graphs
hits = TP
subs = substitution_count
ins = insertion_count
dels = deletion_count

# === Bar Chart: Hits, Subs, Ins, Dels ===
fig1 = go.Figure(data=[
    go.Bar(
        x=["Hits", "Substitutions", "Insertions", "Deletions"],
        y=[hits, subs, ins, dels],
        marker_color=["blue", "#fff176", "#81c784", "#e57373"],
        hoverinfo="y+name"
    )
])

fig1.update_layout(
    title="WER Components (Interactive)",
    yaxis=dict(title='Count', range=[0, max([hits, subs, ins, dels]) + 5]),
    xaxis=dict(title='Components'),
    template='plotly_white'
)

# Generate self-contained HTML string
plot_html1 = fig1.to_html(full_html=False, include_plotlyjs='cdn')

ref_len = len(ground_truth.split())
gen_len = len(generated_text.split())
# === Bar Chart: Word count difference ===
fig2 = go.Figure(data=[
    go.Bar(
        x=["Ground Truth", "Generated"],
        y=[ref_len, gen_len],
        marker_color=["#42a5f5", "#ab47bc"],
        hoverinfo="y+name",
        hovertext=[
            f"Words: {ref_len}<br>{ground_truth}",
            f"Words: {gen_len}<br>{generated_text}"
        ]
    )
])

fig2.update_layout(
    title="Word Count Comparison (Interactive)",
    yaxis=dict(title='Word Count', range=[0, max(ref_len, gen_len) + 5]),
    xaxis=dict(title='Text Type'),
    template='plotly_white'
)

plot_html2 = fig2.to_html(full_html=False, include_plotlyjs='cdn')

# === Bar Chart: WER before vs after normalization ===

fig3 = go.FigureWidget(data=[
    go.Bar(
        name='WER',
        x=['Raw WER', 'Normalized WER'],
        y=[wer_raw, wer_score],
        marker_color=['#ef5350', '#66bb6a']
    )
])

fig3.update_layout(
    title='WER: Raw vs Normalized',
    yaxis=dict(range=[0, 1], title='WER Score'),
    xaxis=dict(title='WER Type'),
    template='plotly_white'
)

# Get HTML as a string (with plotly.js included from CDN)
plot_html = fig3.to_html(full_html=False, include_plotlyjs='cdn')

# === Final HTML ===
html_content = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>WER Report</title>
    <style>
        body {{ font-family: 'Poppins', sans-serif; background: #f0f2f5; margin: 0; padding: 40px; color: #333; }}
        .wer {{ position: fixed; top: 20px; right: 30px; background: linear-gradient(135deg, #007bff, #00c6ff); color: white; padding: 10px 20px; font-size: 22px; border-radius: 10px; box-shadow: 0 4px 10px rgba(0,0,0,0.3); }}
        h2 {{ text-align: center; font-size: 32px; margin-bottom: 40px; }}
        .container {{ display: flex; gap: 30px; flex-wrap: nowrap; max-width: 1200px; margin: 0 auto; }}
        .box {{ background: white; flex: 1; padding: 20px; border-radius: 16px; box-shadow: 0 6px 18px rgba(0,0,0,0.1); min-height: 400px; max-height: 400px; overflow-y: auto; }}
        h3 {{ text-align: center; color: #444; margin-bottom: 20px; }}
        span {{ margin-right: 5px; padding: 4px 8px; border-radius: 6px; display: inline-block; }}
        .substitution {{ background-color: #fff176; }}   /* Yellow */
        .insertion    {{ background-color: #81c784; color: white; }}  /* Green */
        .deletion     {{ background-color: #e57373; color: white; }}  /* Red */
        .error-gray   {{ background-color: #cfd8dc; color: black; }}  /* Gray */
        table {{ width: 100%; border-collapse: collapse; margin-top: 20px; }}
        th, td {{ padding: 12px; border: 1px solid #ccc; }}
        details {{ margin-top: 50px; background: white; padding: 20px; border-radius: 16px; box-shadow: 0 6px 18px rgba(0,0,0,0.1); }}
        summary {{ font-size: 24px; font-weight: bold; cursor: pointer; }}

         .wer-title {{
          text-align: center;
          color: #2c3e50;
          font-size: 28px;
          margin-bottom: 20px;
        }}

        .wer-metrics {{
          display: flex;
          flex-wrap: wrap;
          justify-content: center;
          gap: 20px;
        }}

        .wer-card {{
          background-color: #ecf0f1;
          padding: 20px;
          border-radius: 10px;
          width: 150px;
          text-align: center;
          box-shadow: 2px 2px 8px rgba(0, 0, 0, 0.05);
          transition: transform 0.2s ease;
        }}

        .wer-card:hover {{
          transform: translateY(-4px);
          background-color: #e0eafc;
        }}

        .wer-label {{
          font-size: 16px;
          color: #34495e;
          margin-bottom: 10px;
        }}

        .wer-value {{
          font-size: 22px;
          font-weight: bold;
          color: #2980b9;
        }}

        .metrics-section {{
          font-family: 'Segoe UI', sans-serif;
          margin: 40px 0;
          text-align: center;
        }}

        .metrics-section h2 {{
          font-size: 2em;
          color: #2c3e50;
          margin-bottom: 25px;
        }}

        .card-container {{
          display: flex;
          flex-wrap: wrap;
          justify-content: center;
          gap: 20px;
        }}

        .metric-card {{
          background: #ffffff;
          border-radius: 16px;
          box-shadow: 0 4px 12px rgba(0, 0, 0, 0.1);
          padding: 20px 25px;
          width: 220px;
          text-align: center;
          transition: transform 0.2s ease;
        }}

        .metric-card:hover {{
          transform: translateY(-5px);
        }}

        .metric-card h3 {{
          margin-bottom: 8px;
          font-size: 1.2em;
          color: #34495e;
        }}

        .metric-card p {{
          margin: 5px 0 10px;
          font-size: 0.9em;
          color: #7f8c8d;
        }}

        .metric-value {{
          font-size: 1.6em;
          font-weight: bold;
        }}

  </style>
</head>
<body>
    <div class="wer">WER: {wer_score:.2%}</div>
    <h2>Word Error Rate (WER) Analysis</h2>

    <div class="container">
        <div class="box">
            <h3>Generated Text (Highlighted)</h3>
            <p>
              <span style="color:black; background:white;"></span>
              {highlighted_generated_html}
            </p>
        </div>
        <div class="box">
            <h3>Ground Truth (Highlighted)</h3>
            <p>
              <span style="color:black; background:white;"></span>
              {highlighted_ground_html}
            </p>
        </div>
    </div>

    <details open>
        <summary>WER Comparison</summary>
        {wer_formula_html}
    </details>

    <details open>
        <summary> WER Metrics</summary>
        {confusionmatrix_metrics_html}
    </details>

    <details open>
        <summary> Confusion Matix with Metrics</summary>
        {wer_metrics_html}
        {conf_matrix_html}
    </details>

    <details open>
        <summary>Evaluation Metrics</summary>
        {html_template}
    </details>

    <details open>
        <summary>Visual Representation</summary>
        {img_html + plot_html1 + plot_html2 + plot_html }
    </details>

</body>
</html>
"""

with open("wer_report_full.html", "w", encoding="utf-8") as f:
    f.write(html_content)
print("✅ Full WER HTML report saved as 'wer_report_full.html'")


Name: jiwer
Version: 3.1.0
Summary: Evaluate your speech-to-text system with similarity measures such as word error rate (WER)
Home-page: 
Author: 
Author-email: Nik Vaessen <nikvaes@gmail.com>
License: 
Location: /usr/local/lib/python3.11/dist-packages
Requires: click, rapidfuzz
Required-by: 
['what', 'is', 'rnn', 'recurrent', 'neural', 'networkrnn', 'is', 'a', 'type', 'of', 'neural', 'network', 'where', 'the', 'output', 'from', 'the', 'previous', 'step', 'is', 'fed', 'as', 'input', 'to', 'the', 'current', 'step', 'in', 'some', 'cases', 'when', 'it', 'is', 'required', 'to', 'predict', 'the', 'next', 'word', 'of', 'a', 'sentence', 'the', 'previous', 'words', 'are', 'required', 'and', 'hence', 'there', 'is', 'a', 'need', 'to', 'remember', 'the', 'previous', 'words', 'thus', 'rnn', 'came', 'into', 'existence', 'which', 'solved', 'this', 'issue', 'with', 'the', 'help', 'of', 'a', 'hidden', 'layer', 'the', 'main', 'and', 'most', 'important', 'feature', 'of', 'rnn', 'is', 'its', 'hidden', '

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from jiwer import process_words, Compose, ToLowerCase, RemoveWhiteSpace, RemoveMultipleSpaces, RemovePunctuation, Strip

# === Read files ===
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

# Input text paths (replace these with actual paths as needed)
generated_text = read_file('/content/RES0034_transcription.txt')
ground_truth = read_file('/content/RES0034_text_groundtruth.txt')


# Normalization pipeline
wer_standardize_contiguous = Compose([
    ToLowerCase(),
    RemoveWhiteSpace(replace_by_space=True),
    RemoveMultipleSpaces(),
    RemovePunctuation(),
    Strip(),
])

# Apply normalization
gt_transformed = wer_standardize_contiguous(ground_truth)
gen_transformed = wer_standardize_contiguous(generated_text)

# Alignment using jiwer
alignment = process_words(gt_transformed, gen_transformed)

# Build confusion matrix
labels = sorted(set(gt_transformed.split() + gen_transformed.split()))
label_indices = {label: i for i, label in enumerate(labels)}
matrix = np.zeros((len(labels), len(labels)), dtype=int)

for chunk in alignment.alignments:
    # Check if chunk is a list of AlignmentChunks or a single AlignmentChunk
    if isinstance(chunk, list):
        for sub_chunk in chunk:
            if sub_chunk.type == "equal":
                for i in range(sub_chunk.ref_end_idx - sub_chunk.ref_start_idx):
                    word = gt_transformed.split()[sub_chunk.ref_start_idx + i]
                    idx = label_indices[word]
                    matrix[idx, idx] += 1
            elif sub_chunk.type == "substitute":
                ref_word = gt_transformed.split()[sub_chunk.ref_start_idx]
                hyp_word = gen_transformed.split()[sub_chunk.hyp_start_idx]
                matrix[label_indices[ref_word], label_indices[hyp_word]] += 1
            elif sub_chunk.type == "delete":
                ref_word = gt_transformed.split()[sub_chunk.ref_start_idx]
                matrix[label_indices[ref_word], -1] += 1  # treat -1 as missing
            elif sub_chunk.type == "insert":
                hyp_word = gen_transformed.split()[sub_chunk.hyp_start_idx]
                matrix[-1, label_indices[hyp_word]] += 1  # treat -1 as missing
    else:
        if chunk.type == "equal":
            for i in range(chunk.ref_end_idx - chunk.ref_start_idx):
                word = gt_transformed.split()[chunk.ref_start_idx + i]
                idx = label_indices[word]
                matrix[idx, idx] += 1
        elif chunk.type == "substitute":
            ref_word = gt_transformed.split()[chunk.ref_start_idx]
            hyp_word = gen_transformed.split()[chunk.hyp_start_idx]
            matrix[label_indices[ref_word], label_indices[hyp_word]] += 1
        elif chunk.type == "delete":
            ref_word = gt_transformed.split()[chunk.ref_start_idx]
            matrix[label_indices[ref_word], -1] += 1  # treat -1 as missing
        elif chunk.type == "insert":
            hyp_word = gen_transformed.split()[chunk.hyp_start_idx]
            matrix[-1, label_indices[hyp_word]] += 1  # treat -1 as missing

# Convert matrix to DataFrame
df_cm = pd.DataFrame(matrix, index=labels, columns=labels)

# Normalize for percentages
df_cm_percent = df_cm.div(df_cm.sum(axis=1).replace(0, np.nan), axis=0) * 100

# Plot interactive confusion matrix
fig = px.imshow(
    df_cm_percent,
    labels=dict(x="Predicted", y="Actual", color="Percentage"),
    x=labels,
    y=labels,
    text_auto=".1f",
    color_continuous_scale="Blues",
    title="Interactive Confusion Matrix (Percentages)"
)
fig.update_layout(margin=dict(l=50, r=50, t=50, b=50))

fig.show()
